In [24]:
import os
import numpy as np
from PIL import Image
import tensorflow as tf
from tensorflow.keras.preprocessing import image

def embed_lsb(img_path, payload_ratio=0.5):

    img = Image.open(img_path)
    if img.mode != "RGB":
        img = img.convert("RGB")

    arr = np.array(img, dtype=np.uint8)
    flat_arr = arr.flatten()

    num_pixels = flat_arr.size
    num_bits_to_embed = int(num_pixels * payload_ratio)


    msg_bits = np.random.randint(0, 2, num_bits_to_embed)


    flat_arr[:num_bits_to_embed] = (flat_arr[:num_bits_to_embed] & 0xFE) | msg_bits

    new_arr = flat_arr.reshape(arr.shape)
    return Image.fromarray(new_arr)


model_path = "lsb_mobilenetv2_model (2).h5"
model = tf.keras.models.load_model(model_path)


def predict_lsb(model, img_path, img_size=(224, 224)):

    img = image.load_img(img_path, target_size=img_size, color_mode='rgb')
    img_array = image.img_to_array(img)


    img_array = img_array / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    prob = model.predict(img_array)[0][0]
    pred = 1 if prob > 0.5 else 0
    label = "LSB Embedded" if pred == 0 else "Clean Image"

    print(f"Prediction: {label}")
    return label, prob


clean_image_path = "clean image.jpg"
lsb_test_path = "lsb_test.jpg"


lsb_image = embed_lsb(clean_image_path, payload_ratio=0.5)
lsb_image.save(lsb_test_path)


predict_lsb(model, lsb_test_path)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Prediction: LSB Embedded


('LSB Embedded', np.float32(0.010247117))